### Thermal Image Object Detection

In [1]:
import numpy as np 
import cv2

In [3]:
# Load the pre-trained HOG detector for pedestrian detection
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Read the input image
img = cv2.imread('data/3.png')

# Perform pedestrian detection
pedestrians, _ = hog.detectMultiScale(img)

# Draw rectangles around the detected pedestrians
for (x, y, w, h) in pedestrians:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Pedestrian Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Read the input image
img = cv2.imread('data/2.png')

# Perform pedestrian detection
detected, weights = hog.detectMultiScale(img)

# Draw rectangles around the detected pedestrians
for (x, y, w, h) in detected:
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

# Display the result
cv2.imshow('Pedestrian Detection', img)
# cv2.imshow('Pedestrian Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()